In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import plotly.express as px

df = pd.read_csv('76-16_president.csv')
cols_to_keep = ['year','state','state_po','candidate','party','writein','candidatevotes','totalvotes']
df = df[cols_to_keep]
electoral_votes = pd.read_csv('electoral_votes.csv')
df.head()

,year,state,state_po,candidate,party,writein,candidatevotes,totalvotes
0,1976,Alabama,AL,"Carter, Jimmy",democrat,False,659170,1182850
1,1976,Alabama,AL,"Ford, Gerald",republican,False,504070,1182850
2,1976,Alabama,AL,"Maddox, Lester",american independent party,False,9198,1182850
3,1976,Alabama,AL,"Bubar, Benjamin """"Ben""""",prohibition,False,6669,1182850
4,1976,Alabama,AL,"Hall, Gus",communist party use,False,1954,1182850


In [3]:
%%html
<script>
    // AUTORUN ALL CELLS ON NOTEBOOK-LOAD!
    require(
        ['base/js/namespace', 'jquery'], 
        function(jupyter, $) {
            $(jupyter.events).on("kernel_ready.Kernel", function () {
                console.log("Auto-running all cells-below...");
                jupyter.actions.call('jupyter-notebook:run-all-cells-below');
                jupyter.actions.call('jupyter-notebook:save-notebook');
            });
        }
    );
</script>

In [4]:
def election_results():
    year = input('Please enter an election year from 1976 to 2016: ')
    year = int(year)
    if year not in [1976, 1980, 1984, 1988, 1992, 1996, 2000, 2004, 2008, 2012, 2016]:
        year = input('Please enter a valid election year from 1976 to 2016: ')
        year = int(year)
    print('{} ELECTION ANALYSIS\n'.format(year))
    focus_df = df[df['year']==year]
    democrat = df[(df['year']==year)&(df['party']=='democrat')]['candidate'].unique()[0]
    republican = df[(df['year']==year)&(df['party']=='republican')]['candidate'].unique()[0]
    candidates = [democrat, republican]
    
    democrat_first = democrat.split(',')[1].strip()
    democrat_last = democrat.split(',')[0].strip()
    democrat = democrat_first + ' ' + democrat_last
    
    republican_first = republican.split(',')[1].strip()
    republican_last = republican.split(',')[0].strip()
    republican = republican_first + ' ' + republican_last
    print('Candidates: {} (R) & {} (D)'.format(republican, democrat))

    dem_df = focus_df[focus_df['candidate']==candidates[0]]
    dem_df['votesclean'] = dem_df.groupby(dem_df['state'])['candidatevotes'].transform(sum)
    dem_df = dem_df.drop_duplicates(subset='state_po')
    
    if year==2012:
        rep_df = focus_df[(focus_df['candidate']==candidates[1])|(focus_df['candidate']=='Mitt, Romney')]
    else:
        rep_df = focus_df[focus_df['candidate']==candidates[1]]
        
    rep_df['votesclean'] = rep_df.groupby(rep_df['state'])['candidatevotes'].transform(sum)
    rep_df = rep_df.drop_duplicates(subset='state_po')

    dem_df = dem_df[['year', 'state', 'state_po', 'votesclean', 'totalvotes']]
    dem_df.columns = ['year', 'state', 'state_code', 'votes_d', 'total_votes']
    dem_df.reset_index(inplace=True)
    
    rep_df = rep_df[['year', 'state', 'state_po', 'votesclean', 'totalvotes']]
    rep_df.columns = ['year', 'state', 'state_code', 'votes_r', 'totalvotes']
    rep_df.reset_index(inplace=True)
    
    dem_df['votes_r'] = rep_df['votes_r']
    final_df = dem_df
    final_df['votes_i'] = final_df['total_votes'] - (final_df[['votes_d', 'votes_r']].sum(axis=1))
    final_df = final_df[['year', 'state', 'state_code', 'votes_d', 
       'votes_r', 'votes_i', 'total_votes']]
    
    final_df['electoral_votes'] = electoral_votes[str(year)]
    elec_votes = final_df['electoral_votes'].sum()
    print('Total Electoral Votes in {}: '.format(year), elec_votes)
    print('Electoral Votes Needed to Win Presidency: ', ((elec_votes/2).round(0)+1).astype(int), '\n')
    
    vote_count = final_df[['votes_d', 'votes_r', 'votes_i']].sum()
    vote_count.index = [democrat, republican, 'Independent']
    vote_count = vote_count.sort_values(ascending=False)
    pop_winner = vote_count.nlargest(1).index[0]
    print('Total Votes:', vote_count.to_string(), sep='\n')
    print('Winner of the Popular Vote: ', pop_winner)
    
    final_df['ev_d'] = (final_df['electoral_votes'] * (final_df['votes_d']/final_df['total_votes'])).round(0)
    final_df['ev_r'] = (final_df['electoral_votes'] * (final_df['votes_r']/final_df['total_votes'])).round(0)
    final_df['ev_i'] = (final_df['electoral_votes'] * (final_df['votes_i']/final_df['total_votes'])).round(0)

    final_df['state_winner'] = 'Republican'
    #Change to democrat if they won the state
    mask = final_df['votes_d'] > final_df['votes_r']
    final_df.loc[mask,'state_winner'] = 'Democrat'
    
    for x in (final_df[(final_df['electoral_votes'] != final_df[['ev_d','ev_r','ev_i']].sum(axis=1))==True].index):
        if final_df.loc[x,'electoral_votes']-1 == final_df.loc[x,['ev_d','ev_r','ev_i']].sum():
            if final_df.loc[x, 'state_winner'] == 'Republican':
                final_df.loc[x, 'ev_r']+= 1
            elif final_df.loc[x, 'state_winner'] == 'Democrat':
                final_df.loc[x, 'ev_d']+= 1
        elif final_df.loc[x,'electoral_votes']+1 == final_df.loc[x,['ev_d','ev_r','ev_i']].sum():
            if final_df.loc[x, 'ev_i'] > 0:
                final_df.loc[x, 'ev_i']-= 1
        else:
            print(final_df[(final_df['electoral_votes'] != final_df[['ev_d','ev_r','ev_i']].sum(axis=1))==True].index)

    final_ev = final_df[['ev_d','ev_r','ev_i']].sum().astype(int)
    final_ev.index = [democrat, republican, 'Independent']
    final_ev = final_ev.sort_values(ascending=False)
    winner = final_ev.nlargest(1).index[0]
    print('\nTotal Electoral Votes by Candidate Under Popular-Electoral Method:', final_ev.to_string(), sep='\n')
    print('Winner of the {} Popular-Electoral Presidential Election: '.format(year), winner)
    if final_ev.nlargest(1).values[0] < 270:
        print('*Winner by Majority, Does Not Meet 270 Vote Threshold*')
    print('\nCOMPARE\n')
    
    final_df['old_ev_r'] = 0
    final_df['old_ev_d'] = 0
    for x in final_df[final_df['state_winner']=='Republican'].index:
        final_df.loc[x, 'old_ev_r'] = final_df.loc[x, 'electoral_votes']
    for x in final_df[final_df['state_winner']=='Democrat'].index:
        final_df.loc[x, 'old_ev_d'] = final_df.loc[x, 'electoral_votes']
    old_ev = final_df[['old_ev_d', 'old_ev_r']].sum()
    old_ev.index = [democrat, republican]
    old_ev = old_ev.sort_values(ascending=False)
    print('Actual Electoral Votes in the {} Election:'.format(year), old_ev.to_string(), sep='\n')
    actual_winner = old_ev.nlargest(1).index[0]
    print('Winner of the {} Presidential Election: '.format(year), actual_winner)
    
    final_df['% Democrat'] = (final_df['votes_d']/(final_df['votes_r']+final_df['votes_d'])*100).round(2)
    fig = px.choropleth(final_df,  # Input Pandas DataFrame
                        locations='state_code',  # DataFrame column with locations
                        color='% Democrat',  # DataFrame column with color values
                        hover_name='state', # DataFrame column hover info
                        locationmode = 'USA-states', # Set to plot as US States
                        color_continuous_scale = 'rdbu',
                        range_color=(20,80))

    fig.update_layout(
        title_text = 'States Based on Voting Margins', # Create a Title
        geo_scope='usa',  # Plot only the USA instead of globe
    )
    fig.show(renderer='notebook_connected')
    return final_df


In [5]:
election_results()

Please enter an election year from 1976 to 2016: 1988
1988 ELECTION ANALYSIS

Candidates: George H.W. Bush (R) & Michael Dukakis (D)
Total Electoral Votes in 1988:  538
Electoral Votes Needed to Win Presidency:  270 

Total Votes:
George H.W. Bush    48886097
Michael Dukakis     41809074
Independent           891654
Winner of the Popular Vote:  George H.W. Bush

Total Electoral Votes by Candidate Under Popular-Electoral Method:
George H.W. Bush    291
Michael Dukakis     246
Independent           1
Winner of the 1988 Popular-Electoral Presidential Election:  George H.W. Bush

COMPARE

Actual Electoral Votes in the 1988 Election:
George H.W. Bush    426
Michael Dukakis     112
Winner of the 1988 Presidential Election:  George H.W. Bush


,year,state,state_code,votes_d,votes_r,votes_i,total_votes,electoral_votes,ev_d,ev_r,ev_i,state_winner,old_ev_r,old_ev_d,% Democrat
0,1988,Alabama,AL,549506,815576,13394,1378476,9,4.0,5.0,0.0,Republican,9,0,40.25
1,1988,Alaska,AK,72584,119251,8281,200116,3,1.0,2.0,0.0,Republican,3,0,37.84
2,1988,Arizona,AZ,454029,702541,15303,1171873,7,3.0,4.0,0.0,Republican,7,0,39.26
3,1988,Arkansas,AR,349237,466578,11923,827738,6,3.0,3.0,0.0,Republican,6,0,42.81
4,1988,California,CA,4702233,5054917,129915,9887065,47,22.0,24.0,1.0,Republican,47,0,48.19
5,1988,Colorado,CO,621453,728177,22764,1372394,8,4.0,4.0,0.0,Republican,8,0,46.05
6,1988,Connecticut,CT,676584,750241,16569,1443394,8,4.0,4.0,0.0,Republican,8,0,47.42
7,1988,Delaware,DE,108647,139639,1605,249891,3,1.0,2.0,0.0,Republican,3,0,43.76
8,1988,District of Columbia,DC,159407,27590,5880,192877,3,3.0,0.0,0.0,Democrat,0,3,85.25
9,1988,Florida,FL,1656701,2618885,26727,4302313,21,8.0,13.0,0.0,Republican,21,0,38.75
